### Importación de los módulos

Se importan los módulos necesarios para el entrenamiento del modelo de control

In [1]:
import pandas as pd
import torch
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import dgl.nn as dglnn
from sklearn.model_selection import train_test_split
import networkx as nx
import random
from statistics import mean
import csv
import time
from torch.utils.data import Dataset, DataLoader
from itertools import product

### Función de generación de grafo aleatorio

Esta función genera un grafo completo ponderado con pesos aleatorios usado en lso ejemplos de la evaluación del coste medio de los caminos generados por el modelo

In [2]:
def generate_random_tsp_graph(n_cities, min_distance=1, max_distance=100):
    G = nx.Graph()
    for i in range(n_cities):
        G.add_node(i)

    for i in range(n_cities):
        for j in range(i+1, n_cities):
            distance = random.randint(min_distance, max_distance)
            G.add_edge(i, j, weight=distance)

    return G

def graph_to_adjacency_matrix(G):
    return nx.adjacency_matrix(G).todense().tolist()

Se definen las constantes como el número de clases de la predicción y la longitud de las secuencias.
Se lee el dataset previamente generado en `dataset_generation_processing.ipynb`

In [3]:
SEQUENCE_LENGTH = 14
NUM_CLASSES_OUT = 15

In [4]:
df = pd.read_csv('dataset_full_adj_matrix_15_nodes_stock.csv')

### Función usada para obtener los índices no redundantes de una matriz de adyacencia de tamaño NxN

La función lista los índices de los coeficientes situados bajo la diagonal principal de una matriz

In [5]:
def below_diagonal_indices(n):
    # List to store the indices of elements below the diagonal
    indices = []

    # Loop through each row
    for i in range(1, n):  # Start from 1 because the first row does not have any elements below the diagonal
        # Loop through each column up to (but not including) the diagonal
        for j in range(i):
            # Compute the index in the flattened matrix and add to the list
            index = i * n + j
            indices.append(index)

    return indices

### Obtención de la matriz de adyacencia a partir del dataset

Se leen los coeficientes y se convierten en un tensor que contiene 14000 ejemplos de matrices.

In [6]:
selected_columns = ['col_' + str(i) for i in range(1, NUM_CLASSES_OUT**2 + 1)]
selected_data = df[selected_columns]

numpy_array = selected_data.values
tensor = torch.tensor(numpy_array).view(df.shape[0], NUM_CLASSES_OUT, NUM_CLASSES_OUT)
adjacency_matrices = tensor
adjacency_matrices.shape

torch.Size([14000, 15, 15])

Se utiliza la función descrita con anterioridad para obtener los coeficientes bajo la diagonal principal de cada matriz de adyacencia del tesnor.  
Cada matriz de 15x15 genera 105 predictores correspondientes a los coeficientes no repetidos.

In [7]:
relevant_indices = below_diagonal_indices(NUM_CLASSES_OUT)
flat_matrices = torch.zeros((df.shape[0], (NUM_CLASSES_OUT**2 - NUM_CLASSES_OUT) // 2))
for i in range(df.shape[0]):
    for index, j in enumerate(relevant_indices):
        flat_matrices[i][index] = df[f'col_{j+1}'][i]

### Normalización de las matrices de adyacencia

Se normalizan los coeficientes de las matrices entre -0.5 y 0.5

In [8]:
flat_matrices = flat_matrices / 101
flat_matrices

tensor([[0.9010, 0.5842, 0.9208,  ..., 0.7129, 0.3960, 0.9010],
        [0.9010, 0.5842, 0.9208,  ..., 0.7129, 0.3960, 0.9010],
        [0.9010, 0.5842, 0.9208,  ..., 0.7129, 0.3960, 0.9010],
        ...,
        [0.7723, 0.1683, 0.4851,  ..., 0.7822, 0.8812, 0.9307],
        [0.7723, 0.1683, 0.4851,  ..., 0.7822, 0.8812, 0.9307],
        [0.7723, 0.1683, 0.4851,  ..., 0.7822, 0.8812, 0.9307]])

In [9]:
graph_indices = flat_matrices
graph_indices = graph_indices - 0.5
graph_indices.shape

torch.Size([14000, 105])

### Obtención de las secuencias de nodos
Se obtienen los caminos parciales que componen la parte secuencial de los problemas de clasificación, se leen del dataframe y se convierten en enteros.  
Se forma un tensor de 14.000 ejemplos con las secuencias obtenidas.

In [10]:
sequence_tensor = torch.zeros((df.shape[0], NUM_CLASSES_OUT - 1))
for i in range(df.shape[0]):
    for j in range(SEQUENCE_LENGTH):
        sequence_tensor[i][j] = df[f'seq_{j+1}'][i]

sequence_tensor = sequence_tensor.long()
sequence_tensor

tensor([[ 0, -1, -1,  ..., -1, -1, -1],
        [ 0, 12, -1,  ..., -1, -1, -1],
        [ 0, 12,  4,  ..., -1, -1, -1],
        ...,
        [ 0,  2, 10,  ...,  7, -1, -1],
        [ 0,  2, 10,  ...,  7, 14, -1],
        [ 0,  2, 10,  ...,  7, 14,  8]])

### Codificación de las secuencias
Se utiliza una combinación de advanced indexing y tensores booleanos (masks) para codificar cada elemento (nodo) de la secuencia con su fila correspondiente en la matriz de adyacencia del ejemplo (grafo) en el que aparece.

In [11]:
batch_indices = torch.arange(len(sequence_tensor)).view(-1, 1).expand_as(sequence_tensor)
mask = (sequence_tensor == -1)

sequence_tensor = sequence_tensor.masked_fill(mask, 0)

sequence_encodings = adjacency_matrices[batch_indices, sequence_tensor]
sequence_encodings = sequence_encodings.masked_fill(mask.unsqueeze(-1), 0)

sequence_tensor = sequence_tensor.masked_fill(mask, -1)

In [12]:
sequence_encodings = sequence_encodings / 101
sequence_encodings[sequence_encodings != 0] -= 0.5

### Extracción de los objetivos
Se crea un tensor leyendo los objetivos almacenados en el dataset.

In [13]:
targets_tensor = torch.tensor(df['next_node'])
targets_tensor.shape

torch.Size([14000])

### Definicion del modelo
El modelo RNN consiste en: 
1. Una capa Linear que procesa los coeficientes de las matrices de adyacencia.
2. Una capa Red Neuronal Recurrente de la clase RNN de Pytorch que procesa las secuencias de nodos que componen los caminos parciales.
3. Una capa Linear que toma las salidas de las capas anteriores como entrada y devuelve la clasificación final.

In [14]:
class HybridModel(nn.Module):
    def __init__(self, hidden_size, num_layers, dropout):
        super(HybridModel, self).__init__()

        self.num_layers = num_layers
        self.hidden_size = hidden_size

        # Adjacency matrix processing layer
        self.matrix_proc = nn.Linear((NUM_CLASSES_OUT**2 - NUM_CLASSES_OUT) // 2, NUM_CLASSES_OUT)
        
        # Recurrent Neural Network (RNN)
        self.rnn = nn.RNN(input_size=NUM_CLASSES_OUT, hidden_size=hidden_size, num_layers=num_layers, batch_first=True, dropout=dropout)# , dropout=0.1)

        # Fully Connected Layer
        self.fc = nn.Linear(NUM_CLASSES_OUT + hidden_size, NUM_CLASSES_OUT)

    def forward(self, flat_matrix, sequence_encoding):
        # Process adjacency matrix through Adjacency matrix processing layer
        matrix_out = self.matrix_proc(flat_matrix)

        # Process input through RNN
        rnn_output, _ = self.rnn(sequence_encoding)  # [batch_size, seq_len, num_classes]
        rnn_output = rnn_output[:, -1, :]

        # Concatenate outputs from adjacency matrix and RNN
        concat_output = torch.cat((matrix_out, rnn_output), dim=1)
        
        # Feed concatenated output through Fully Connected Layer
        out = self.fc(concat_output)
        return out


### Dataset & Dataloader
Se crea una clase hija de la clase Dataset de Pytorch para almacenar los datos.  
Esto permite mayor flexibilidad para situar la codificación de los datos en distintas partes del código y ha sido extremadamente útil durante el desarrollo de este trabajo.  
Actualmente la implementación del experimento podría hacerse con la clase Dataset directamente pero se deja implementada esta clase por si fuese útil para la continuación futura de la investigación.

In [15]:
class CustomGraphSequenceDataset(Dataset):
    def __init__(self, sequence_tensor, adj_matrices, targets_tensor):
        self.sequence_tensor = sequence_tensor
        self.adj_matrices = adj_matrices
        self.targets_tensor = targets_tensor

    def __len__(self):
        return len(self.sequence_tensor)

    def __getitem__(self, idx):
        sequence = self.sequence_tensor[idx]
        adj_matrix = self.adj_matrices[idx]
        target = self.targets_tensor[idx]
        return sequence, adj_matrix, target

De forma similar a la clase anterior, esta función collate personalizada es extremadamente útil a la hora de hacer pruebas con las diversas posibilidades para transformar los grupos de ejemplos en batches.  
Queda en el código por si viese más utilidad en la investigación futura, a pesar de que actualmente los scripts funcionarían con la función collate por defecto de pytorch.

In [16]:
def custom_collate_fn(batch):
    sequences, graphs, targets = zip(*batch)
    # Convert sequences to a single tensor
    sequences = torch.stack(sequences)
    # Convert targets to a single tensor
    targets = torch.stack(targets)
    # Batch graphs into a single graph
    batched_graph = torch.stack(graphs) # dgl.batch(graphs)
    return sequences, batched_graph, targets

### Train Test split
Se dividen las codificaciones de las secuencias, las matrices de adyacencia y los objetivos entre conjuntos de entrenamiento y validación.
Se crean los datasets de entrenamiento y test a partir de los tensores resultantes.

In [17]:
# Split the data into training and testing sets (80% train, 20% test)
train_sequences, test_sequences, train_adj_matrices, test_adj_matrices, train_targets, test_targets = train_test_split(
    sequence_encodings, graph_indices, targets_tensor, test_size=0.2, random_state=42)

In [18]:
# Create Custom Dataset for train and test
train_dataset = CustomGraphSequenceDataset(train_sequences, train_adj_matrices, train_targets)
test_dataset = CustomGraphSequenceDataset(test_sequences, test_adj_matrices, test_targets)

### Evaluación del coste medio de los caminos
Esta función recibe un modelo como parámetro y se ejecuta durante la fase de evaluación para calcular el coste medio del camino generado por este modelo.

Esta función genera un batch de grafos completos ponderados con pesos aleatorios.  
Una vez generados, se obtienen las matrices de adyacencia correspondientes a los grafos y se procesan sus coeficientes tal como se hace en el apartado anterior (Obtención de la matriz de adyacencia a partir del dataset)..

Se generan tensores correspondientes a secuencias de nodos vacías (todas tienen 0 como nodo inicial como en los ejemplos del dataset).
Se codifican dichas secuencias y se generan los elementos siguientes a partir de predicciones del modelo.


In [19]:
def path_evaluation(model, batch_size):
    matrices_tensor_batch = torch.zeros((batch_size, (NUM_CLASSES_OUT**2 - NUM_CLASSES_OUT) // 2))
    adjacency_matrices = np.ndarray((batch_size, NUM_CLASSES_OUT, NUM_CLASSES_OUT))

    # Generate one batch of adjacency matrices 
    for tensor_row in range(batch_size):
        graph = generate_random_tsp_graph(NUM_CLASSES_OUT)
        adjacency_matrix = graph_to_adjacency_matrix(graph)
        adjacency_matrices[tensor_row] = torch.tensor(adjacency_matrix).view((NUM_CLASSES_OUT, NUM_CLASSES_OUT))
        
        partial_matrix = list()
        for relevant_index in relevant_indices:
            flattened_adj_matrix = [item for sublist in adjacency_matrix for item in sublist]
            partial_matrix.append(flattened_adj_matrix[relevant_index])
        
        matrices_tensor_batch[tensor_row] = torch.tensor(partial_matrix)

    # Normalize
    matrices_tensor_batch = matrices_tensor_batch / 101

    # Generate an empty sequence from a random starting point
    sequences_tensor_batch = torch.zeros((batch_size, NUM_CLASSES_OUT - 1)).long()
    sequences_tensor_batch[:, 0] = torch.tensor(np.random.randint(0, 15, size=(batch_size)))

    # Store the decision cost in each step of the way
    costs = torch.zeros((NUM_CLASSES_OUT - 2, batch_size))
    for i in range(NUM_CLASSES_OUT - 2):
        # Encode the sequence for the model
        batch_indices = torch.arange(len(sequences_tensor_batch)).view(-1, 1).expand_as(sequences_tensor_batch)
        mask = (sequences_tensor_batch == -1)

        sequences_tensor_batch = sequences_tensor_batch.masked_fill(mask, 0)

        sequence_encodings = torch.tensor(adjacency_matrices[batch_indices, sequences_tensor_batch])
        sequence_encodings = sequence_encodings.masked_fill(mask.unsqueeze(-1), 0)

        sequences_tensor_batch = sequences_tensor_batch.masked_fill(mask, -1)

        sequence_encodings = sequence_encodings / 101
        sequence_encodings[sequence_encodings != 0] -= 0.5
        
        sequence_encodings = sequence_encodings.to(torch.float32)

        outputs = model(matrices_tensor_batch, sequence_encodings)
        _, predicted_labels = torch.max(outputs, 1)
        previous_labels = sequences_tensor_batch[:, i]
        costs_this_timestep = list()
        for batch_index in range(batch_size):
            matrix = adjacency_matrices[batch_index]
            # print(len(matrix))
            # print(predicted_labels[batch_index])
            # print(previous_labels[batch_index])
            costs_this_timestep.append(matrix[predicted_labels[batch_index]][previous_labels[batch_index]])

        costs_this_timestep = torch.tensor(costs_this_timestep)
        costs[i] = costs_this_timestep

    # Calculate total cost of each path
    costs = torch.sum(costs, dim=0)

    # Return the mean value from all the path costs
    return torch.mean(costs).item()

### Instanciamiento y loop
La siguiente funcion instancia un modelo con los hiperparámetros pasados como argumentos, y lo entrena durante 100 épocas.
Para ello utiliza el algoritmo de optimización Adam y la función de pérdida CrossEntropyLoss.  
Se crea un archivo de registro con los resultados del modelo en el directorio results, que contiene en el nombre los hiperparámetros usados.  
Durante la fase de evaluación, además de comprobar la precisión del modelo sobre el conjunto de test, se evalúa el coste medio en los caminos generados por este sobre un batch de ejemplos generados aleatoriamente.

In [20]:
def train_model(train_loader, test_loader, hyperparams, identifier, batch_size):
    num_layers = hyperparams['num_layers']
    hidden_size = hyperparams['hidden_size']
    learning_rate = hyperparams['learning_rate']
    dropout = hyperparams['dropout']
    decay = hyperparams['decay']


    # Initialize new model, optimizer and loss function
    model = HybridModel(num_layers=num_layers, hidden_size=hidden_size, dropout=dropout)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=decay)#, weight_decay=1e-5)

    # Number of epochs
    num_epochs = 100
    total_path_costs_mean = list()


    csv_file = open(f"./results/rnn/{batch_size}/{identifier}.csv", mode="a", newline="")
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(["Epoch", "Train_Accuracy", "Test_Accuracy", "Mean_Path_Cost", "Elapsed_Time"])

    # Training loop
    start_time = time.time()
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        total_samples = 0
        correct_predictions = 0

        batch_num = len(train_loader)
        for batch_idx, (sequences, batched_graph, targets) in enumerate(train_loader):
            
            # Zero the gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(batched_graph, sequences)
            
            # Compute the loss
            loss = criterion(outputs, targets)
            
            loss.backward()

            optimizer.step()

            # Update the training accuracy
            _, predicted_labels = torch.max(outputs, 1)
            total_samples += targets.size(0)
            correct_predictions += (predicted_labels == targets).sum().item()
        
        # Compute the training accuracy for the epoch
        accuracy = correct_predictions / total_samples

        # Evaluation phase
        model.eval()
        total_samples = 0
        correct_predictions = 0
        mean_this_epoch = 0

        with torch.no_grad():
            for sequences, batched_graph, targets in test_loader:
                # Forward pass
                outputs = model(batched_graph, sequences)

                # Update the testing accuracy
                _, predicted_labels = torch.max(outputs, 1)
                total_samples += targets.size(0)
                correct_predictions += (predicted_labels == targets).sum().item()

            # Path evaluation
            mean_this_epoch = path_evaluation(model, batch_size)
            total_path_costs_mean.append(mean_this_epoch)

        # Compute the testing accuracy for the epoch
        test_accuracy = correct_predictions / total_samples

        elapsed_time = time.time() - start_time
        csv_writer.writerow([
            epoch + 1,
            round(accuracy * 100, 4),
            round(test_accuracy * 100, 4),
            round(mean_this_epoch, 4),
            round(elapsed_time, 4)
        ])
        csv_file.flush()
        
        print(f'Epoch [{epoch + 1}/{num_epochs}]')
        print(f'Train Accuracy: {accuracy * 100:.2f}%, Testing Accuracy: {test_accuracy * 100:.2f}%')
        print(f'Path cost mean this epoch: {mean_this_epoch:.2f}, Total path cost mean : {mean(total_path_costs_mean):.2f}\n')

### Grid de hiperparámetros
En esta última función se especifican los distintos valores que tendrán los hiperparámetros del grid, y se entrena al modelo previamente definido utilizando todas las combinaciones posibles de hiperparámetros para ello.

In [21]:
batch_sizes =  (16, 32, 64)

hyperparameter_grid = {
    'learning_rate': [0.0001, 0.0005, 0.001],
    'hidden_size': [16, 32, 64],
    'num_layers': [1, 2, 3],
    'dropout': [0.1, 0.2, 0.3],
    'decay': [1e-5, 5e-5, 1e-4]
}

for batch_size in batch_sizes:
    print('current batch size is ' + str(batch_size))

    all_combs = product(*(hyperparameter_grid[hp] for hp in hyperparameter_grid))
    for comb in all_combs:
        hyperparams = dict(zip(hyperparameter_grid.keys(), comb))

        identifier = '_'.join([f"{k}={v:.0e}" if k == 'decay' else f"{k}={v}" for k, v in hyperparams.items()])    
            
        print(f"Training with {identifier}")
        
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=custom_collate_fn, drop_last=True)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=custom_collate_fn, drop_last=True)
        
        train_model(train_loader, test_loader, hyperparams, identifier, batch_size)

current batch size is 64
Training with learning_rate=0.0005_hidden_size=64_num_layers=2_dropout=0_decay=0e+00


/tmp/ipykernel_462/1631721209.py:14: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  return nx.adjacency_matrix(G).todense().tolist()


Epoch [1/250]
Train Accuracy: 6.84%, Testing Accuracy: 6.98%
Path cost mean this epoch: 482.50, Total path cost mean : 482.50

Epoch [2/250]
Train Accuracy: 7.25%, Testing Accuracy: 7.85%
Path cost mean this epoch: 375.72, Total path cost mean : 429.11

Epoch [3/250]
Train Accuracy: 7.64%, Testing Accuracy: 6.72%
Path cost mean this epoch: 394.92, Total path cost mean : 417.71

Epoch [4/250]
Train Accuracy: 8.96%, Testing Accuracy: 7.38%
Path cost mean this epoch: 323.28, Total path cost mean : 394.11

Epoch [5/250]
Train Accuracy: 9.28%, Testing Accuracy: 7.81%
Path cost mean this epoch: 322.97, Total path cost mean : 379.88

Epoch [6/250]
Train Accuracy: 9.92%, Testing Accuracy: 7.70%
Path cost mean this epoch: 321.56, Total path cost mean : 370.16

Epoch [7/250]
Train Accuracy: 10.46%, Testing Accuracy: 8.43%
Path cost mean this epoch: 277.81, Total path cost mean : 356.97

Epoch [8/250]
Train Accuracy: 10.97%, Testing Accuracy: 8.94%
Path cost mean this epoch: 280.36, Total path co